In [4]:
import pandas as pd
import numpy as np

In [5]:
# Read votation data
df = pd.read_pickle("data/votations.pkl")

In [6]:
# Function that capitalizes the first letter in a given string
def cap_first(s):
    return s[0].capitalize() + s[1:]

In [7]:
# Create a dataframe with the indices of the votation data
data = pd.DataFrame([x for x, _ in df.index.values]).drop_duplicates()

# Rename the community column
data.columns = ["Commune"]

# Create columns for districts/cantons/countries
data["District"] = np.nan
data["Canton"] = np.nan
data["Pays"] = np.nan

# Extract names of districts/cantons/countries
data["Pays"] = data["Commune"].map(lambda x : x if x[0] != ">" and x[0] != "-" and x[0] != "." else np.nan)
data["Canton"] = data["Commune"].map(lambda x : x[2:] if x[0] == "-" else np.nan)
data["District"] = data["Commune"].map(lambda x : x[3:] if x[0] == ">" else np.nan)

# Propagate names of districts/cantons/countries downwards
data = data.fillna(method='ffill')

# Remove lines that do not describe a community
data = data[data["Commune"].map(lambda x : x[0] == ".")]

# Clean canton and district names
data["Canton"] = data["Canton"].map(lambda x : x if x is np.nan else x.split(" /")[0])
data["District"] = data["District"].map(lambda x :
                                        x if \
                                            "Bezirk See" in x else \
                                        "".join(x.split("'")[1:]).strip() if \
                                            "District d'" in x or \
                                            "District de l'" in x else
                                        cap_first(" ".join(x.split(" ")[2:])).strip() if \
                                            "Arrondissement administratif" in x or \
                                            "District" in x or \
                                            "Canton" in x or \
                                            "Distretto di" in x else \
                                        " ".join(x.split(" ")[1:]).strip() if \
                                            "Verwaltungskreis" in x or \
                                            "Wahlkreis" in x or \
                                            "Kanton" in x or \
                                            "Bezirk" in x or \
                                            "Region" in x \
                                        else x)

data["District"] = data["District"].map(lambda x : \
                                        "Obwald" if x == "Obwalden" else \
                                        "Nidwald" if x == "Nidwalden" else \
                                        x)

# Write correct district/canton/country data for foreign votes
data[["District", "Canton", "Pays"]] = data.apply(lambda x : pd.Series(["-", "-", "Etranger"]) if \
                                        "-Ausland-" in x["District"] or \
                                        " de l'étranger" in x["District"] or \
                                        "-Korrespondenzweg" in x["District"] or \
                                        "-autres" in x["District"] or \
                                        "-voto per corrispondenza" in x["District"] \
                                      else pd.Series([x["District"], x["Canton"], x["Pays"]]), axis=1)

#data.set_index('Commune', inplace=True)
data = data.reset_index(drop=True)

#NAMES NEED TO BE CLEANED AFTER MODIFYING THE PROPER DATA, OTHERWISE COMMUNITY NAMES WILL NOT MATCH
#NO DISTRICTS FOR GENEVA, SCHAFFHAUSEN, APPENZELL INNERRHODEN, OBWALD AND NIDWALD

In [8]:
# Create columns for districts/cantons/countries

df = pd.read_pickle("data/votations.pkl")

df.reset_index(inplace=True)
df = df.merge(data, on="Commune")

In [31]:
themes = pd.read_csv("data/px-x-1703010000_103.csv", sep=";", encoding="cp1254", skiprows=2)[:-1]
themes = themes[~themes['Période'].str.contains("bis")]
themes["Période"] = themes["Période"].apply(lambda x : x.split(" ")[1])
themes = themes.set_index("Période")
themes.head()

,Régime politique,Politique étrangère,Politique de sécurité,Economie,Finances publiques,"Infrastructure, aménagement, environnement",Politique sociale,"Enseignement, culture et médias"
Période,,,,,,,,
2017,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2016,1.0,0.0,0.0,1.0,0.0,5.0,6.0,0.0
2015,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0
2014,1.0,1.0,1.0,1.0,2.0,1.0,5.0,0.0
2013,1.0,0.0,1.0,1.0,0.0,2.0,6.0,0.0


,Votation,Année
0,14.06.1981 Protection des consommateurs,1981
1,29.11.1981 Régime financier,1981
2,14.06.1981 Egalité entre hommes et femmes,1981
3,06.06.1982 Code pénal suisse,1982
4,06.06.1982 Loi sur les étrangers,1982


In [58]:
#We extract the subjects of votations from the index of the data and make a dataframe we the subjects and the year
df = pd.read_pickle("data/votations.pkl")
votations = pd.DataFrame(df.index.levels[1])
votations["Année"] = votations["Votation"].apply(lambda s : s[6:10])
votations = votations.sort_values("Année")
votations = votations.reset_index(drop=True)

header = list(themes.columns)
values = []

#Now let's use our indexation of themes to append a theme to each votation. We use a handmade indexation of the
#themes since no mapping between subjects and themes exist online. We used the available listing of voted themes
#for each year and manually reattributed the themes to the subjects.
with open("data/theme_indices.txt", "r") as file:
    for year in votations["Année"].unique():
        indices = file.readline().replace(" ", "").split(",")
        temp = []
        
        #We get the themes for the current year
        for i in range(len(header)):
            for j in range(int(themes.loc[year, header[i]])):
                temp.append(header[i])

        #We reorder them using our indexation and add them to the list
        values += list(map(lambda x : temp[int(x) - 1], indices))
        
#We add the list to the dataframe as a column
votations["Thématique"] = values
votations

,Votation,Année,Thématique
0,14.06.1981 Protection des consommateurs,1981,Economie
1,29.11.1981 Régime financier,1981,Finances publiques
2,14.06.1981 Egalité entre hommes et femmes,1981,Politique sociale
3,06.06.1982 Code pénal suisse,1982,Régime politique
4,06.06.1982 Loi sur les étrangers,1982,Politique sociale
5,28.11.1982 Surveillance des prix (initiative),1982,Economie
6,28.11.1982 Surveillance des prix (contre-projet),1982,Economie
7,04.12.1983 Naturalisation facilitée,1983,Régime politique
8,04.12.1983 Droit de la nationalité,1983,Régime politique
9,27.02.1983 Article sur l'énergie,1983,"Infrastructure, aménagement, environnement"


In [12]:
df["Commune"] = df["Commune"].map(lambda x : x[7:] if x[0] == "." else x)
# data["Commune"] = data["Commune"].map(lambda x : x.split(" (")[0])

df

KeyError: 'Commune'